In [ ]:
!unzip -q /content/drive/MyDrive/atlas_vision_shotquality_sportsradar.zip -d /content
%cd /content/atlas_vision_shotquality_sportsradar
!ls


/content/atlas_vision_shotquality_sportsradar
atlas_vision  config  notebooks  README.md  requirements.txt  run_pipeline.py


In [ ]:

!pip install -q -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.3 MB/s eta 0:00:00


In [ ]:
!apt-get -y install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
%%writefile keys.txt
SPORTSRADAR_API_KEY=XYZ

Writing keys.txt


In [ ]:
!mkdir -p data

In [ ]:
!ls data

In [ ]:
import requests

def load_sportsradar_key(keys_file="keys.txt"):
    with open(keys_file, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            if "=" in line:
                _, val = line.split("=", 1)
                return val.strip()
            return line
    raise RuntimeError("No valid key in keys.txt")

api_key = load_sportsradar_key()
print("Loaded key prefix:", api_key[:4] + "****")


Loaded key prefix: nEZd****


In [ ]:
access_level = "trial"
lang = "en"

def get_daily_schedule(year, month, day):
    url = f"https://api.sportradar.com/nba/{access_level}/v8/{lang}/games/{year}/{month:02d}/{day:02d}/schedule.json"
    params = {"api_key": api_key}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

year, month, day = 2016, 12, 25
sched = get_daily_schedule(year, month, day)
games = sched.get("games", [])
print("Number of games:", len(games))


Number of games: 5


In [ ]:
home_filter = "Cavaliers"
away_filter = "Warriors"

for g in games:
    gid   = g["id"]
    home  = g["home"]["name"]
    away  = g["away"]["name"]
    h_alias = g["home"]["alias"]
    a_alias = g["away"]["alias"]
    scheduled = g.get("scheduled", "")
    home_pts  = g.get("home_points")
    away_pts  = g.get("away_points")

    print("GAME ID:", gid)
    print(f"  Home: {home} ({h_alias}), Away: {away} ({a_alias})")
    print(f"  Scheduled: {scheduled}")
    print(f"  Final score: {home_pts} - {away_pts}")
    print("-" * 60)

print("\nMatches for Cavs vs Warriors:")
for g in games:
    gid   = g["id"]
    home  = g["home"]["name"]
    away  = g["away"]["name"]
    if home_filter.lower() in home.lower() and away_filter.lower() in away.lower():
        print("MATCH:", gid, ":", away, "@", home)


GAME ID: 6f069693-e89c-4b97-86bf-6d089112a566
  Home: New York Knicks (NYK), Away: Boston Celtics (BOS)
  Scheduled: 2016-12-25T17:00:00+00:00
  Final score: 114 - 119
------------------------------------------------------------
GAME ID: d56d8b51-dc9f-404c-987a-8ba49fc2c854
  Home: Cleveland Cavaliers (CLE), Away: Golden State Warriors (GSW)
  Scheduled: 2016-12-25T19:30:00+00:00
  Final score: 109 - 108
------------------------------------------------------------
GAME ID: c7d9b4b0-2665-40ae-8d29-7b9879cb9a5e
  Home: San Antonio Spurs (SAS), Away: Chicago Bulls (CHI)
  Scheduled: 2016-12-25T22:00:00+00:00
  Final score: 119 - 100
------------------------------------------------------------
GAME ID: f7906368-2423-49f2-a2fe-49dce12bf2b5
  Home: Oklahoma City Thunder (OKC), Away: Minnesota Timberwolves (MIN)
  Scheduled: 2016-12-26T01:00:00+00:00
  Final score: 112 - 100
------------------------------------------------------------
GAME ID: bc31fdee-d7ec-4f8f-a049-3ab449acdfea
  Home: Los 

d56d8b51-dc9f-404c-987a-8ba49fc2c854

In [ ]:
import yaml

cfg_path = "config/game_config.yaml"

with open(cfg_path) as f:
    cfg = yaml.safe_load(f)

cfg["sportsradar_game_id"] = "d56d8b51-dc9f-404c-987a-8ba49fc2c854"

with open(cfg_path, "w") as f:
    yaml.safe_dump(cfg, f)

!cat config/game_config.yaml


cached_model_path: null
frame_height_fraction_max: 0.98
frame_height_fraction_min: 0.3
homography_npy: config/homography.npy
kaggle_zip_path: data/shotquality-rebounding.zip
keys_file: keys.txt
max_players_on_court: 10
min_player_detections: 5
output_csv_path: outputs/game_shotquality_cv.csv
pbp_path: data/game_pbp_mapped.csv
scoreboard_roi:
- 0.35
- 0.02
- 0.65
- 0.12
sportsradar_access_level: trial
sportsradar_game_id: d56d8b51-dc9f-404c-987a-8ba49fc2c854
sportsradar_language: en
tipoff_video_time_sec: null
video_path: data/game.mp4
work_dir: outputs/work
yolo_model_name: yolov8n.pt


upload game.mp4 and shotquality zip before running

In [ ]:
!mkdir -p data
!cp '/content/drive/MyDrive/game.zip' ./data/
!ls -lh data

total 867M
-rw------- 1 root root 867M Dec  1 19:59 game.zip


In [ ]:
!unzip -o data/game.zip -d data
!ls -lh data

Archive:  data/game.zip
  inflating: data/game.mp4           
total 1.7G
-rw-rw-rw- 1 root root 871M Nov 30 22:51 game.mp4
-rw------- 1 root root 867M Dec  1 19:59 game.zip
-rw-r--r-- 1 root root 2.0M Dec  1 19:59 shotquality-rebounding.zip


In [ ]:
!python run_pipeline.py --config config/game_config.yaml

Mapped PBP not found; fetching from SportsRadar and aligning to video ...
Using configured tip-off time: t=35.50 seconds.
Saved mapped PBP with video_time_sec to data/game_pbp_mapped.csv
Loaded 98 missed shots from mapped PBP.
Using homography: [[   0.073438           0 -1.2307e-14]
 [          0    -0.11574      83.333]
 [          0           0           1]]
[WARN] Shot 9780ae3e-0125-4db4-a1c9-cb311801b348: only 1 player detections, skipping.
[WARN] Shot ace8b2ba-7ebb-41e2-92fe-d5d11ea9b6e4: only 1 player detections, skipping.
[WARN] Shot b8555e60-f916-432e-a34b-a9b341ab0799: only 1 player detections, skipping.
[WARN] Shot ff7926c0-4496-4060-a89d-b1985a94a4b5: only 0 player detections, skipping.
[WARN] Shot 70b652e4-5bc5-4993-a79c-753036380c1d: only 2 player detections, skipping.
Saved NBA locs to outputs/work/nba_locs.csv
Saved NBA features to outputs/work/nba_features_raw.csv
[OREB] GBM validation AUC:   0.6011
[OREB] GBM validation Brier: 0.1977
[OREB] Calibrated AUC:       0.6011